In [ ]:

import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose
from hidden_points_removal import hidden_point_removal_o3d
import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics.metrics_class import Metrics
import shutil
from sam3d_util import * 
from point_cloud_utils import transform_pcd, get_pcd, change_point_indices, get_statistical_inlier_indices, angle_between, get_subpcd, kDTree_1NN_feature_reprojection
from image_utils import masks_to_image
from hidden_points_removal import hidden_point_removal_o3d
from point_to_pixels import point_to_pixel
from point_cloud_utils import merge_chunks_unite_instances2, remove_semantics
import umap
import copy
import cv2
import scipy
from merged_sequences import merged_sequence

Here we define the dataset depending on kitti sequence!

In [ ]:
DATASET_PATH = os.path.join('/media/cedric/Datasets2/semantic_kitti/')
end_inds = {0:4541,2:4661,3:800,5:2761,6:1101,7:1100,8:4071,9:1591,10:1201}
SEQUENCE_NUM = 3

ind_start = 0
ind_end = end_inds[SEQUENCE_NUM]
minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 15 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False

out_chunks = '../../pcd_preprocessed/output_chunks/'

out_folder_ncuts = out_chunks + 'sam3d/' + str(SEQUENCE_NUM) + '/'
if os.path.exists(out_folder_ncuts) == False : 
        os.makedirs(out_folder_ncuts)

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)

out_folder = '../../pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Now we aggregate a large point cloud based on (ind_start, ind_end)
## This cell can be ignored after first run as outputs are stored 

In [ ]:
if os.path.exists(out_folder + 'all_poses_' + str(SEQUENCE_NUM) + '_' + str(0) + '.npz') == False:
        process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                                major_voxel_size=major_voxel_size,icp=False,
                                out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                                ground_segmentation_method=ground_segmentation_method)

This cell can be ignored after first run as outputs are stored 

In [ ]:
##load data if already stored 

if os.path.exists(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd') == False:
        pcd_ground_minor, pcd_nonground_minor,\
                all_poses, T_pcd, first_position,kitti_labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                        ground_mode=ground_segmentation_method)
        #o3d.visualization.draw_geometries([color_pcd_by_labels(pcd_nonground_minor,kitti_labels['seg_nonground'])])
        o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=False)
        o3d.io.write_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_0.pcd', pcd_nonground_minor, write_ascii=False, compressed=False, print_progress=False)
        np.savez(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_0.npz',
                                                instance_nonground=kitti_labels['instance_nonground'],
                                                instance_ground=kitti_labels['instance_ground'],
                                                seg_ground = kitti_labels['seg_ground'],
                                                seg_nonground=kitti_labels['seg_nonground']
                                                )


In [ ]:
pcd_ground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd')
pcd_nonground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_0.pcd')
print(pcd_ground_minor)
kitti_labels_orig = {}
with np.load(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_0.npz') as data :
        kitti_labels_orig['instance_ground'] = data['instance_ground']
        kitti_labels_orig['instance_nonground'] = data['instance_nonground']
        kitti_labels_orig['seg_nonground'] = data['seg_nonground']
        kitti_labels_orig['seg_ground'] = data['seg_ground']

        

with np.load(f'{out_folder}all_poses_{SEQUENCE_NUM}_0.npz') as data:
        all_poses = data['all_poses']
        T_pcd = data['T_pcd']
        first_position = T_pcd[:3, 3]

Now we subsample the poses based on a voxel_size

In [ ]:
poses, positions, \
sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start)

Now we can split the point cloud into chunks based on a tbd chunk_size

In [ ]:
pcd_nonground_chunks, pcd_ground_chunks,\
pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
indices,indices_ground, center_positions, \
center_ids, chunk_bounds, kitti_labels, obbs = chunk_and_downsample_point_clouds(dataset,pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                            first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                            overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=kitti_labels_orig)

In [ ]:
def color_pcd_by_labels(pcd, labels,colors=None,gt_labels=None):
    
    if colors == None : 
        colors = generate_random_colors(2000)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colors = np.zeros(np.asarray(pcd.points).shape)
    if gt_labels is None :
    	unique_labels = list(np.unique(labels)) 
    else: 
        unique_labels = list(np.unique(gt_labels))
    background_color = np.array([0,0,0])


    #for i in range(len(pcd_colored.points)):
    for i in unique_labels:
        if i == -1 : 
            continue
        idcs = np.where(labels == i)
        idcs = idcs[0]
        if i == 0 : 
            pcd_colors[idcs] = background_color
        else : 
            pcd_colors[idcs] = np.array(colors[unique_labels.index(i)])
        
        #if labels[i] != (-1):
        #    pcd_colored.colors[i] = np.array(colors[labels[i]]) / 255
    pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors/ 255)
    return pcd_colored

def masks_to_image(masks):
    '''
    Function that takes an array of masks and returns an pixel-wise label map
    '''
    image_labels = np.zeros(masks[0]['segmentation'].shape)
    for i, mask in enumerate(masks):
        image_labels[mask['segmentation']] = i + 1
 
    return image_labels

In [ ]:
from sam3d_util import * 
from tqdm import tqdm 
out_kitti = out_chunks + 'out_kitti/'
if os.path.exists(out_kitti) == True : 
        shutil.rmtree(out_kitti)

os.makedirs(out_kitti)
        


limit = -1 ##use this for experiments to run limit chunks numberss


patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
out_data = []


for sequence in tqdm(range(0,len(center_ids)-1)):

                print('id',center_ids[sequence])
                inliers = get_statistical_inlier_indices(pcd_ground_chunks[sequence])
                ground_inliers = get_subpcd(pcd_ground_chunks[sequence], inliers)
                mean_hight = np.mean(np.asarray(ground_inliers.points)[:,2])
                in_idcs = np.where(np.asarray(ground_inliers.points)[:,2] < (mean_hight + 0.2))[0]
                cut_hight = get_subpcd(ground_inliers, in_idcs)
                cut_hight.paint_uniform_color([0, 0, 0])
        
                '''
                seg_dict, pts = ncuts_chunk(dataset,indices,pcd_nonground_chunks,pcd_ground_chunks,
                        pcd_nonground_chunks_major_downsampling,
                        pcd_nonground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,sampled_indices_global,
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=False,sequence=sequence,
                        patchwise_indices=patchwise_indices,ncuts_threshold=ncuts_threshold)
                '''
                
                out_pcd1 = sam3d(dataset,indices, pcd_nonground_minor,
                        T_pcd, sampled_indices_global,patchwise_indices=patchwise_indices,sequence=sequence,pcd_chunk=pcd_nonground_chunks[sequence])
                
                #out_pcd = our_sam3d(dataset,patchwise_indices[sequence][0],center_ids[sequence],pcd_nonground_chunks[sequence])
                #o3d.visualization.draw_geometries([out_pcd1])
                #o3d.visualization.draw_geometries([out_pcd])
                
                name =  str(center_ids[sequence]).zfill(6) + '.pcd'
                o3d.io.write_point_cloud(out_folder_ncuts + name, out_pcd1 + cut_hight, write_ascii=False, compressed=False, print_progress=False)

In [ ]:
point_clouds_dbscan = get_merge_pcds(out_folder_ncuts)

merge_kitti_instance = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti_instance' + str(SEQUENCE_NUM) + '.pcd')
merge_dbscan = merge_chunks_unite_instances2(point_clouds_dbscan)
o3d.io.write_point_cloud(out_folder + 'sam3d_ours' + str(SEQUENCE_NUM) + '.pcd',merge_dbscan)




unique_colors, labels_dbscan = np.unique(np.asarray(merge_dbscan.colors), axis=0, return_inverse=True)
unique_colors, labels_kitti = np.unique(np.asarray(merge_kitti_instance.colors),axis=0, return_inverse=True)


print(labels_kitti.shape,labels_dbscan.shape)
new_dbscan_labels = remove_semantics(labels_kitti,labels_dbscan)


metrics_sam3d = Metrics(name='sam3d')


metrics_sam3d.update_stats(labels_dbscan,new_dbscan_labels,labels_kitti,calc_all=True)





In [ ]:
o3d.visualization.draw_geometries([merge_dbscan])